# ETL

---

**Author:** Diego Antonio García Padilla

**Date:** Oct 29, 2025

## Enviroment setup

In [7]:
#@title Setup & Environment Verification

import warnings
warnings.filterwarnings('ignore')

import os
import sys

print("=== ENVIRONMENT CHECK ===")
print(f"Python: {sys.version.split()[0]}")
print(f"JAVA_HOME: {os.environ.get('JAVA_HOME')}")
print(f"SPARK_HOME: {os.environ.get('SPARK_HOME')}")
print(f"Driver Memory: {os.environ.get('SPARK_DRIVER_MEMORY')}")
print(f"Executor Memory: {os.environ.get('SPARK_EXECUTOR_MEMORY')}")
print("=" * 50)

=== ENVIRONMENT CHECK ===
Python: 3.10.12
JAVA_HOME: /usr/lib/jvm/java-8-openjdk-arm64/jre
SPARK_HOME: /opt/spark
Driver Memory: 8g
Executor Memory: 4g


In [8]:
#@title Import Libraries

# PySpark
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import *
from pyspark.sql.window import Window

# Data manipulation
import pandas as pd
import numpy as np

# Financial data
import yfinance as yf

# Hugging Face
from huggingface_hub import hf_hub_download

# Kaggle
import kagglehub

# Utilities
from datetime import datetime, timedelta
import json
import requests
import logging
from tqdm import tqdm
import time
import subprocess
from pathlib import Path

In [9]:
#@title Start Spark session

import os
import subprocess
from pyspark.sql import SparkSession

print("=== PRE-FLIGHT CHECK ===")

# Verify Java is available
try:
    java_version = subprocess.check_output(['java', '-version'], stderr=subprocess.STDOUT)
    print("Java: ✅ Available")
except Exception as e:
    print(f"Java: ❌ Not available - {e}")

print("=" * 50)

# Create fresh Spark session
spark = SparkSession.builder \
    .appName("SAPS_Sentiment_Analysis") \
    .master("local[8]") \
    .config("spark.driver.memory", "8g") \
    .config("spark.executor.memory", "4g") \
    .config("spark.driver.maxResultSize", "4g") \
    .config("spark.sql.shuffle.partitions", "100") \
    .config("spark.sql.adaptive.enabled", "true") \
    .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer") \
    .config("spark.kryoserializer.buffer.max", "1024m") \
    .config("spark.local.dir", "/tmp/spark") \
    .getOrCreate()

spark.sparkContext.setLogLevel("WARN")

print(f"\n✅ Spark {spark.version} initialized successfully")
print(f"   Master: {spark.sparkContext.master}")
print(f"   App Name: {spark.sparkContext.appName}")
print(f"   Driver Memory: 8GB")
print(f"   Spark UI: http://localhost:4040")

=== PRE-FLIGHT CHECK ===
Java: ✅ Available

✅ Spark 3.5.6 initialized successfully
   Master: local[8]
   App Name: SAPS_Sentiment_Analysis
   Driver Memory: 8GB
   Spark UI: http://localhost:4040


## Download data

### Amazon Reviews

In [10]:
#@title Download Yelp reviews dataset

yelp_path = kagglehub.dataset_download("yelp-dataset/yelp-dataset")

print("Path to dataset files:", yelp_path)

def explore_dataset(path):
    print(f"📁 Content of: {path}\n")
    total_size = 0
    
    for root, dirs, files in os.walk(path):
        for file in files:
            file_path = os.path.join(root, file)
            size = os.path.getsize(file_path)
            total_size += size
            size_mb = size / (1024 * 1024)
            print(f"  📄 {file}")
            print(f"     Size: {size_mb:.2f} MB")
    
    total_gb = total_size / (1024 * 1024 * 1024)
    print(f"\n💾 Total size: {total_gb:.2f} GB ({total_size / (1024 * 1024):.2f} MB)")
    
    return total_size

explore_dataset(yelp_path)

Path to dataset files: /root/.cache/kagglehub/datasets/yelp-dataset/yelp-dataset/versions/4
📁 Content of: /root/.cache/kagglehub/datasets/yelp-dataset/yelp-dataset/versions/4

  📄 Dataset_User_Agreement.pdf
     Size: 0.08 MB
  📄 yelp_academic_dataset_business.json
     Size: 113.36 MB
  📄 yelp_academic_dataset_checkin.json
     Size: 273.67 MB
  📄 yelp_academic_dataset_review.json
     Size: 5094.40 MB
  📄 yelp_academic_dataset_tip.json
     Size: 172.24 MB
  📄 yelp_academic_dataset_user.json
     Size: 3207.52 MB

💾 Total size: 8.65 GB (8861.26 MB)


9291705417

## Data Explotation

In [11]:
#@title Load Yelp Reviews as Spark dataset

# Parquet path
parquet_path = "../data/raw/yelp_reviews_raw.parquet"

if os.path.exists(parquet_path):
    print(f"✅ Already exists: {parquet_path} \n")
    df_reviews = spark.read.parquet(parquet_path)
    print(f"🔄 Loaded \n")
else:
    # Load the dataset
    reviews_file = os.path.join(yelp_path, "yelp_academic_dataset_review.json")

    # Read JSON file with Spark
    df_reviews = spark.read.json(reviews_file)

# Show schema to understand structure
print("📋 Schema of Yelp Reviews:")
df_reviews.printSchema()

# Basic statistics
print(f"\n📊 Total reviews: {df_reviews.count():,}")

# Show sample data
print("\n🔍 Sample reviews:")
df_reviews.show(5, truncate=50)

# Check stars distribution
print("\n⭐ Stars distribution:")
df_reviews.groupBy('stars').count().orderBy('stars').show()

# Check text lengths

print("\n📝 Text statistics:")
df_reviews.select(
    F.avg(F.length(F.col('text'))).alias('avg_length'),
    F.min(F.length(F.col('text'))).alias('min_length'),
    F.max(F.length(F.col('text'))).alias('max_length')
).show()

if not os.path.exists(parquet_path):
    df_reviews.write.parquet(parquet_path, mode="overwrite")
    print(f"\n💾 Parquet saved: {parquet_path}")

✅ Already exists: ../data/raw/yelp_reviews_raw.parquet 

🔄 Loaded 

📋 Schema of Yelp Reviews:
root
 |-- business_id: string (nullable = true)
 |-- cool: long (nullable = true)
 |-- date: string (nullable = true)
 |-- funny: long (nullable = true)
 |-- review_id: string (nullable = true)
 |-- stars: double (nullable = true)
 |-- text: string (nullable = true)
 |-- useful: long (nullable = true)
 |-- user_id: string (nullable = true)


📊 Total reviews: 6,990,280

🔍 Sample reviews:
+----------------------+----+-------------------+-----+----------------------+-----+--------------------------------------------------+------+----------------------+
|           business_id|cool|               date|funny|             review_id|stars|                                              text|useful|               user_id|
+----------------------+----+-------------------+-----+----------------------+-----+--------------------------------------------------+------+----------------------+
|znK6tMeegKf9YnGut

+-----------------+----------+----------+
|       avg_length|min_length|max_length|
+-----------------+----------+----------+
|567.7644364746477|         1|      5000|
+-----------------+----------+----------+



In [12]:
#@title Sample dataset

# Parquet path
parquet_path = "../data/filtered/yelp_reviews_sentiment.parquet"

if os.path.exists(parquet_path):
    print(f"✅ Already exists: {parquet_path}")
    df_sentiment = spark.read.parquet(parquet_path)
    print(f"🔄 Loaded")
else:
    #  Sample directly from the original dataframe with stars
    # 10% sample = ~700K reviews (still >1GB when processed with text)
    df_sample = df_reviews.sample(fraction=0.20, seed=42)

    # Create sentiment column
    df_sentiment = df_sample.select(
        F.col('review_id'),
        F.col('text'),
        F.col('stars'),
        F.col('useful'),
        F.col('date')
    ).withColumn('sentiment',
        F.when(F.col('stars').isin([1.0, 2.0]), 'negative')
        .when(F.col('stars') == 3.0, 'neutral')
        .when(F.col('stars').isin([4.0, 5.0]), 'positive')
    )

# Single count operation
total_reviews = df_sentiment.count()
print(f"\n✅ Sample dataset created: {total_reviews:,} reviews")

# Get distribution (single pass)
print("\n🎯 Sentiment distribution:")
sentiment_counts = df_sentiment.groupBy('sentiment').count().collect()
for row in sentiment_counts:
    percentage = (row['count'] / total_reviews) * 100
    print(f"   {row['sentiment']}: {row['count']:,} ({percentage:.1f}%)")

# Show one sample per sentiment (lightweight)
print("\n📌 Sample reviews:")
for sent in ['negative', 'neutral', 'positive']:
    sample = df_sentiment.filter(F.col('sentiment') == sent).select('text', 'stars').first()
    if sample:
        print(f"\n{sent.upper()} ({sample['stars']} stars):")
        print(f"   {sample['text'][:150]}...")

df_sentiment.write.parquet(parquet_path, mode="overwrite")
print(f"\n💾 Parquet saved: {parquet_path}")


✅ Sample dataset created: 1,397,232 reviews

🎯 Sentiment distribution:
   positive: 936,506 (67.0%)
   neutral: 138,512 (9.9%)
   negative: 322,214 (23.1%)

📌 Sample reviews:

NEGATIVE (1.0 stars):
   What has the world come when you have to argue with staff in order to get corn salsa on your burrito bowl at a Mexican restaurant? Was repeatedly told...

NEUTRAL (3.0 stars):
   My 91-yr-old mom and I stayed here for a weekend in June while visiting family and attending a concert downtown. My overall reaction to our stay was t...

POSITIVE (5.0 stars):
   Love this local hangout. Not many bells or whistles but it has a great whisky selection and a solid crew of bar tenders manning all 24 hours of day an...



💾 Parquet saved: ../data/filtered/yelp_reviews_sentiment.parquet


## Data Cleaning

In [13]:
#@title Select relevant features

df_filtered = df_sentiment.select("text", "sentiment") \
                .dropDuplicates()

df_filtered.show(10)

+--------------------+---------+
|                text|sentiment|
+--------------------+---------+
|I've been here fo...| positive|
|Absolute nightmar...| negative|
|Not sure who dump...| negative|
|**READ BEFORE BUY...| negative|
|I happened on to ...| positive|
|Had an amazing ex...| positive|
|I was looking for...| negative|
|Do NOT go here an...| negative|
|My first time try...| negative|
|All I can say is ...| negative|
+--------------------+---------+
only showing top 10 rows



In [14]:
#@title Clean text

# Add text length
df_clean = df_filtered.withColumn('text_length', F.length(F.col('text')))

# Add word count
df_clean = df_clean.withColumn('word_count', 
    F.size(F.split(F.col('text'), ' ')))

# Statistics by sentiment
print("\n📊 Text statistics by sentiment:")
df_clean.groupBy('sentiment').agg(
    F.avg('text_length').alias('avg_length'),
    F.avg('word_count').alias('avg_words'),
    F.count('*').alias('count')
).show()

# Clean text: lowercase, remove special characters
df_clean = df_clean.withColumn('text_clean',
    F.lower(F.regexp_replace(F.col('text'), '[^a-zA-Z0-9\\s]', ''))
)

df_clean.select('text', 'text_clean', 'sentiment').show(3, truncate=80)


📊 Text statistics by sentiment:


+---------+-----------------+------------------+------+
|sentiment|       avg_length|         avg_words| count|
+---------+-----------------+------------------+------+
| positive|501.6060316978746| 92.66772888873696|936088|
|  neutral|668.9137329664854|125.21409331513537|138477|
| negative|717.3848549682797| 135.7392441147336|322033|
+---------+-----------------+------------------+------+



+--------------------------------------------------------------------------------+--------------------------------------------------------------------------------+---------+
|                                                                            text|                                                                      text_clean|sentiment|
+--------------------------------------------------------------------------------+--------------------------------------------------------------------------------+---------+
|I've been here for happy hour! I love the cocktail drinks/names. Everything i...|ive been here for happy hour i love the cocktail drinksnames everything is ph...| positive|
|Absolute nightmare. It started to rain so they shut everything down except fo...|absolute nightmare it started to rain so they shut everything down except for...| negative|
|Not sure who dumps out hot coffee 45 minutes prior to closing their doors. Al...|not sure who dumps out hot coffee 45 minutes pri